In [ ]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time
import timm
from timm.models.layers.classifier import ClassifierHead
import torch
import torch.utils.data as data
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import wandb


In [ ]:
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'
df = pd.read_csv(df_path)
project_name = 'wandb_bc'
wandb_dir = './model'


In [ ]:
def get_img_stats(img_dir, img_ids):
    img_info = dict(heights=[], widths=[], means=[], stds=[])
    for img_id in tqdm(img_ids):
        for path in glob(os.path.join(img_dir, img_id, '*')):
            img = np.array(Image.open(path))
            h, w, _ = img.shape
            img_info['heights'].append(h)
            img_info['widths'].append(w)
            img_info['means'].append(img.mean(axis=(0,1)))
            img_info['stds'].append(img.std(axis=(0,1)))
    return img_info

In [ ]:
# img_info = get_img_stats(img_dir, df.path.values)

# print(f'RGB Mean: {np.mean(img_info["means"], axis=0) / 255.}')
# print(f'RGB Standard Deviation: {np.mean(img_info["stds"], axis=0) / 255.}')

In [ ]:
mean, std = ( 0.5601,0.5241,0.5014),(0.2332,0.2430,0.2456)

In [ ]:
from albumentations import *
from albumentations.pytorch import ToTensorV2


def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Resize(img_size[0], img_size[1]),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [ ]:
### 마스크 여부, 성별, 나이를 mapping할 클래스를 생성합니다.

class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2

class GenderLabels:
    male = 0
    female = 1

class AgeGroup:
    map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 60 else 2

In [ ]:
class MaskBaseDataset(data.Dataset):
    num_classes = 3 * 2 * 3

    _file_names = {
        "mask1.jpg": MaskLabels.mask,
        "mask2.jpg": MaskLabels.mask,
        "mask3.jpg": MaskLabels.mask,
        "mask4.jpg": MaskLabels.mask,
        "mask5.jpg": MaskLabels.mask,
        "incorrect_mask.jpg": MaskLabels.incorrect,
        "normal.jpg": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []

    def __init__(self, img_dir, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.img_dir = img_dir
        self.mean = mean
        self.std = std
        self.transform = transform

        self.setup()

    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        profiles = os.listdir(self.img_dir)
        for profile in profiles:
            for file_name, label in self._file_names.items():
                img_path = os.path.join(self.img_dir, profile, file_name)  # (resized_data, 000004_male_Asian_54, mask1.jpg)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    self.mask_labels.append(label)

                    id, gender, race, age = profile.split("_")
                    gender_label = getattr(GenderLabels, gender)
                    age_label = AgeGroup.map_label(age)

                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        
        # 레이블을 불러옵니다.
        mask_label = self.mask_labels[index]
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
        multi_class_label = int(mask_label) * 6 + int(gender_label) * 3 + int(age_label)
        
        # 이미지를 Augmentation 시킵니다.
        image_transform = self.transform(image=np.array(image))['image']
        return image_transform, multi_class_label

    def __len__(self):
        return len(self.image_paths)

In [ ]:
# 정의한 Augmentation 함수와 Dataset 클래스 객체를 생성합니다.
transform = get_transforms(mean=mean, std=std)

dataset = MaskBaseDataset(
    img_dir=img_dir
)

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
train_dataset.dataset.set_transform(transform['train'])
val_dataset.dataset.set_transform(transform['val'])

In [ ]:
train_loader = data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=4,
    shuffle=True
)

valid_loader = data.DataLoader(
    val_dataset,
    batch_size=32,
    num_workers=4,
    shuffle=False
)

In [ ]:
from torchvision import transforms

# Augmentation으로 이미지를 Normalize했기 때문에, 역으로 다시 Normalize 해주어야합니다.
# inv_normalize = transforms.Normalize(
#     mean=[-m / s for m, s in zip(mean, std)],
#     std=[1 / s for s in std]
# )

# n_rows, n_cols = 4, 3

# fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=(16, 24))
# for i in range(n_rows*n_cols):
#     axes[i%n_rows][i//(n_cols+1)].imshow(inv_normalize(images[i]).permute(1, 2, 0))
#     axes[i%n_rows][i//(n_cols+1)].set_title(f'Label: {labels[i]}', color='r')
# plt.tight_layout()

In [ ]:
#이제 모델링 

model_name = "res2next50"
model = timm.create_model('res2next50', pretrained=True)

from timm.models.layers.classifier import ClassifierHead

class MyModel(nn.Module):
    def __init__(self,model_name,pretrained=True):
        super().__init__()
        self.model=timm.create_model(model_name,pretrained=True)
        n_features=self.model.num_features
        
        self.mask_classifier=ClassifierHead(n_features,3)
        self.gender_classifier=ClassifierHead(n_features,2)
        self.age_classifier=ClassifierHead(n_features,3)
    def forward(self,x):
        x=self.model.forward_features(x)
        
        mask=self.mask_classifier(x).view(x.size(0),3,1,1)
        gender=self.gender_classifier(x).view(x.size(0),1,2,1)
        age=self.age_classifier(x).view(x.size(0),1,1,3)
        return (mask*gender*age).view(x.size(0),-1)
        
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

In [ ]:
initial_lr = 0.001
num_epoch =30
batch_size = 50
betas = (0.9, 0.999)
weight_decay = 1e-4


In [ ]:
import torch.nn.functional as F
from sklearn.metrics import f1_score
import torch.optim as optim
from novograd import *
import time

In [ ]:
import torch.nn.init as init

def initialize_weights(model):
    """
    Initialize all weights using xavier uniform. 
    For more weight initialization methods, check https://pytorch.org/docs/stable/nn.init.html
    """
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()

In [ ]:
import pprint as p
net = MyModel(model_name,pretrained=True)

net.to(device)


In [ ]:
optimizer = NovoGrad(net.parameters(), lr=initial_lr, betas=betas, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, 
                                                steps_per_epoch=10, epochs=10,anneal_strategy='linear')
num_epochs = 30
config={'epochs':num_epochs,'batch_size':batch_size,'learning_rate':initial_lr}
wandb.init(project=project_name,config=config,dir=wandb_dir,)
valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

for e in range(num_epochs) :
    p.pprint(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []
    epoch_f1 = 0
    n_iter = 0

    # train
    net.train()
    for i, (images, targets) in enumerate(train_loader) : 
        optimizer.zero_grad()
        
        images = images.to(device)
        targets = targets.to(device)
        
        scores = net(images)
        
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()

        correct = sum(targets == preds).cpu()
        acc=(correct/images.shape[0] * 100)
        epoch_f1 += f1_score(preds.cpu().numpy(), targets.cpu().numpy(), average='macro')
        n_iter += 1

        train_loss_list.append(loss)
        train_acc_list.append(acc)
        
        if i % 50 == 0 :
            p.pprint(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")
    wandb.log({'train loss' : train_mean_loss/(i+1), 'train_accuracy' : float(train_mean_acc)/(i+1),'train_f1 score':f1_score(targets.cpu(),preds.cpu(),average='macro')})
    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)
    
    epoch_f1 = epoch_f1/n_iter

    epoch_time = time.time() - since
    since = time.time()

    p.pprint('')
    p.pprint(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    p.pprint(f'Train Loss Mean %.4f | Accuracy %2.2f | F1-Score %2.4f' % (train_mean_loss, train_mean_acc, epoch_f1) )

    # validation 
    net.eval()
    epoch_f1 = 0
    n_iter = 0
    valid_loss_list = []
    valid_acc_list = []
    
    for i, (images, targets) in enumerate(valid_loader) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)
        
        with torch.no_grad():
            scores = net(images)  
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)
            epoch_f1 += f1_score(preds.cpu().numpy(), targets.cpu().numpy(), average='macro')
            n_iter += 1

        correct = sum(targets == preds).cpu()
        acc=(correct/images.shape[0] * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)


    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")
    wandb.log({'valid_loss' : val_mean_loss/(i+1), 'valid_accuracy' : float(val_mean_acc)/(i+1),'valid_f1 score':f1_score(targets.cpu(),preds.cpu(),average='macro')})

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)
    
    epoch_f1 = epoch_f1/n_iter
    
    p.pprint(f'Valid Loss Mean %.4f | Accuracy %2.2f | F1-Score %2.4f' % (val_mean_loss, val_mean_acc, epoch_f1) )
    p.pprint('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        best_model = net
        path = './model/'
        torch.save(best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            p.pprint("EARLY STOPPING!!")
            break

    lr_scheduler.step()

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        super(TestDataset).__init__()
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [ ]:

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
testimage_dir = os.path.join(test_dir, 'images')

In [ ]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(testimage_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    
    transforms.ToTensor(),
    
])
test_dataset = TestDataset(image_paths, transform)

test_loader = DataLoader(
    test_dataset,
    shuffle=False
)

In [ ]:
best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = best_model(images)
        preds = scores.argmax(dim=-1)
        all_predictions.extend(preds.cpu().numpy())

In [ ]:
from collections import Counter
Counter(all_predictions)

In [ ]:
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_RES2NEXT_step_weight_init.csv'), index=False)
print('test inference is done!')

In [ ]:
from datetime import datetime
from pytz import timezone
print(datetime.now(timezone('asia/seoul')).strftime('%Y-%m-%d %H:%M:%S'))
